### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:


pd.DataFrame([purchase_data["SN"].nunique()], columns = ["Total Players"])


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# create a dataframe and find the number of unique items
data_file_df = pd.DataFrame([purchase_data["Item Name"].nunique()], columns = ["Number of Unique Items"])
#number_of_unique_items = purchase_data["Item Name"].nunique()], columns = ["Number of Unique Items"]

# find the average price
average = purchase_data["Price"].mean()
data_file_df["Average Price"] = average

# Calculate the number of purchases
number_of_purchases = purchase_data["Purchase ID"].count()
data_file_df["Number of Purchases"] = number_of_purchases

# Calucate the total revenues
total_revenues = purchase_data["Price"].sum()
data_file_df["Total Revenues"] = total_revenues

# format columns
format_dict = {col_name: '${:,.2f}' for col_name in data_file_df.select_dtypes(float).columns}

data_file_df.head().style.format(format_dict)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenues
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# filter for unique players
unique_players_df = purchase_data.drop_duplicates(subset=["SN"])

# Get a count of the males, females undisclosed
count = unique_players_df["Gender"].value_counts()

# Create a dataframe and add the count
gender_df = pd.DataFrame({"Total Counts": (count)})

# Get a total gender count
total = unique_players_df["Gender"].count()

# Find the percent of each gender
perc_gender_total = count/total

# Add the percents to the dataframe
gender_df["Percent of Players"] = perc_gender_total

# format columns
format_dict = {col_name: '{:.2%}' for col_name in gender_df.select_dtypes(float).columns}

gender_df.head().style.format(format_dict)

,Total Counts,Percent of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# groupby person and sum the total price
grouped_person_df = purchase_data.groupby(['SN', 'Gender'], as_index=False)['Price'].sum()

# Using GroupBy in order to separate the data into fields according to "Gender" values
grouped_gender_df = purchase_data.groupby(['Gender'])

# In order to be visualized, a data function must be used...
grouped_gender_price_df = grouped_gender_df.describe()['Price']

# filter on 'count' and 'mean'
grouped_gen_price_filt_df = grouped_gender_price_df[['count', 'mean']].copy()

# groupby gender and the sum pri
grouped_gen_price_filt_df['Total Purchase Value'] = purchase_data.groupby(['Gender'])['Price'].sum()

# groupby gender and then take the mean
grouped_gen_price_filt_df["Avg Total Purchase per Person"] = grouped_person_df.groupby(['Gender'])['Price'].mean()

# rename 'count' to 'Purchase Count' and 'mean' to 'Average Purchase Price'
grouped_gen_price_filt_df.rename(columns={'count': 'Purchase Count', 'mean': 'Average Purchase Price'}, inplace=True)

# format 'Purchase Count' to integer
grouped_gen_price_filt_df['Purchase Count'] = grouped_gen_price_filt_df['Purchase Count'].astype(int)

# format all columns to dollars with two decimal points
format_dict = {col_name: '${:,.2f}' for col_name in grouped_gen_price_filt_df.select_dtypes(float).columns}

# print the dataframe with the appropriate format
grouped_gen_price_filt_df = grouped_gen_price_filt_df.head().style.format(format_dict)

# print the dataframe with the appropriate format
grouped_gen_price_filt_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# rename the original dataframe to one for age and drop duplicates
age_demog_df = pd.DataFrame(purchase_data).drop_duplicates(subset=["SN"])

# Establish bins for ages   
bins = [0, 9.5, 14.5, 19.5, 24.5, 29.5, 34.5, 39.5, 50]

# Create the names for the five bins
age_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Categorize the existing players using the age bins. Hint: use pd.cut()
age_demog_df["Age Demographics Summary"] = pd.cut(age_demog_df["Age"], bins, labels=age_names, include_lowest=True)

# Calculate the numbers and percentages by age group
age_demog_groupby_df = age_demog_df.groupby("Age Demographics Summary")

# use a data function to visualize the data
age_demog_groupby_df = age_demog_groupby_df.describe()

list(age_demog_groupby_df.columns)

# use a data function to visualize the data
#age_demog_groupby_df = age_demog_groupby_df.reindex(columns=columns)

# filter so it is only count
age_demog_groupby_df = age_demog_groupby_df[[('Purchase ID', 'count')]].astype(int)

# get the total count
total = age_demog_groupby_df[('Purchase ID', 'count')].sum()

# rename the header to 'Total Count' from 'Purchase ID, count'
age_demog_groupby_df.rename(columns={'Purchase ID': 'Total Count', 'count': ''}, inplace=True)

# get the percent for each age group and add to the dataframe
perc_gender_total = age_demog_groupby_df/total

# Add the percents to the dataframe
age_demog_groupby_df["Percentage of Players"] = perc_gender_total

# format all columns to dollars with two decimal points
format_dict = {col_name: '{:,.2%}' for col_name in age_demog_groupby_df.select_dtypes(float).columns}

# print the dataframe with the appropriate format
age_demog_groupby_df = age_demog_groupby_df.style.format(format_dict)

# Display Age Demographics Table
age_demog_groupby_df

,Total Count,Percentage of Players
,,
Age Demographics Summary,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
# rename the original dataframe to one for age
age_demog_df = purchase_data
# Establish bins for ages   
bins = [0, 9.5, 14.5, 19.5, 24.5, 29.5, 34.5, 39.5, 50]

# Create the names for the five bins
age_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
age_demog_df['Age Ranges'] = pd.cut(age_demog_df['Age'], bins, labels=age_names, include_lowest=True)

# Calculate the numbers and percentages by age group
age_demog_groupby_df = age_demog_df.groupby('Age Ranges')

# Create a summary dataframe to hold the results
age_demo_summary_df = age_demog_groupby_df.describe()

# filter for the 'count' and average purchase price
age_demo_summary_df = age_demo_summary_df[[('Purchase ID', 'count'), ('Price', 'mean')]]

# groupby gender and then sum price
age_demo_summary_df['Total Purchase Value'] = age_demog_df.groupby(['Age Ranges'])['Price'].sum()

# for each age range, divide 'Total Purchase Value' by the total number of unique players
UniqueSN = age_demog_groupby_df['SN'].value_counts()
UniqueSN_groupby_df = UniqueSN.count(0)
age_demo_summary_df['Avg Total Purchase per Person'] = age_demo_summary_df['Total Purchase Value']/UniqueSN_groupby_df

# rename ('Purchase', 'count') to 'Purchase Count' and ('Price', 'mean') to 'Average Purchase Price'
age_demo_summary_df.rename(columns={'Purchase ID': 'Purchase Count', 'count': '', 'Price': 'Average Purchase Price', 'mean': ''}, inplace=True)

# format 'Purchase Count' to integer
age_demo_summary_df[('Purchase Count', '')] = age_demo_summary_df[('Purchase Count', '')].astype(int)

# format all columns to dollars with two decimal points
format_dict = {col_name: '${:,.2f}' for col_name in age_demo_summary_df.select_dtypes(float).columns}

# print the dataframe with the appropriate format
age_demo_summary_df = age_demo_summary_df.style.format(format_dict)


# Display the summary data frame
age_demo_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
,,,,
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
# rename the original dataframe to one for age
SN_groupby_df1 = purchase_data

# group by SN
SN_groupby_df = SN_groupby_df1.groupby('SN')

# Create a summary dataframe to hold the results
SN_groupby_summary_df = SN_groupby_df.describe()

# filter for the 'count' and average purchase price
SN_groupby_summary_df = SN_groupby_summary_df[[('Purchase ID', 'count'), ('Price', 'mean')]]

# groupby gender and the sum pri
SN_groupby_summary_df['Total Purchase Value'] = SN_groupby_df1.groupby(['SN'])['Price'].sum()

# Sort in descending order based on "Total Purchase Value" column
SN_groupby_summary_df = SN_groupby_summary_df.sort_values(('Total Purchase Value'), ascending=False)

# rename 'count' to 'Purchase Count' and 'mean' to 'Average Purchase Price'
#age_demo_summary_df.rename(columns={'count': 'Purchase Count', 'mean': 'Average Purchase Price'}, inplace=True)

# format 'Purchase Count' to integer
SN_groupby_summary_df[('Purchase ID', 'count')] = SN_groupby_summary_df[('Purchase ID', 'count')].astype(int)

# format all columns to dollars with two decimal points
format_dict = {col_name: '${:,.2f}' for col_name in SN_groupby_summary_df.select_dtypes(float).columns}

# print the dataframe with the appropriate format
SN_groupby_summary_df = SN_groupby_summary_df.head().style.format(format_dict)

# Display the summary data frame
SN_groupby_summary_df


,Purchase ID,Price,Total Purchase Value
,count,mean,
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
# Retrieve the Item ID, Item Name, and Item Price columns
grouped_popular_df = purchase_data[['Item ID', 'Item Name', 'Price']]

# Group by Item ID and Item Name
grouped_popular_df = grouped_popular_df.groupby(['Item ID', 'Item Name'])

# Perform calculations to obtain purchase count, average item price, and total purchase value and 
    # create a summary data frame to hold the results
grouped_popular_describe_df = grouped_popular_df['Price'].describe()
grouped_popular_filter_df = grouped_popular_describe_df[['count', 'mean']].copy()
grouped_popular_filter_df["Total Purchase Value"] = grouped_popular_df['Price'].sum()


# Sort in descending order based on "Total Purchase Value" column
grouped_popular_filter_sortCnt_df = grouped_popular_filter_df.sort_values("count", ascending=False)

# rename 'count' to 'Purchase Count' and 'mean' to 'Average Purchase Price'
grouped_popular_filter_sortCnt_df.rename(columns={'count': 'Purchase Count', 'mean': 'Item Price'}, inplace=True)

# format 'Purchase Count' to integer
grouped_popular_filter_sortCnt_df[('Purchase Count')] = grouped_popular_filter_sortCnt_df[('Purchase Count')].astype(int)

# format all columns to dollars with two decimal points
format_dict = {col_name: '${:,.2f}' for col_name in grouped_popular_filter_sortCnt_df.select_dtypes(float).columns}

# print the dataframe with the appropriate format
grouped_popular_filter_sortCnt_df = grouped_popular_filter_sortCnt_df.head().style.format(format_dict)

# Display a preview of the summary data frame
grouped_popular_filter_sortCnt_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
# Sort the above table by total purchase value in descending order
grouped_popular_filter_sortTPV_df = grouped_popular_filter_df.sort_values("Total Purchase Value", ascending=False)


# Optional: give the displayed data cleaner formatting
# rename 'count' to 'Purchase Count' and 'mean' to 'Average Purchase Price'
grouped_popular_filter_sortTPV_df.rename(columns={'count': 'Purchase Count', 'mean': 'Item Price'}, inplace=True)

# format 'Purchase Count' to integer
grouped_popular_filter_sortTPV_df[('Purchase Count')] = grouped_popular_filter_sortTPV_df[('Purchase Count')].astype(int)

# format all columns to dollars with two decimal points
format_dict = {col_name: '${:,.2f}' for col_name in grouped_popular_filter_sortTPV_df.select_dtypes(float).columns}

# print the dataframe with the appropriate format
grouped_popular_filter_sortTPV_df = grouped_popular_filter_sortTPV_df.head().style.format(format_dict)

# Display a preview of the data frame
grouped_popular_filter_sortTPV_df



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
